In [ ]:
import pandas as pd
import os
import re
from glob import glob

from functools import reduce

In [ ]:
path = "/Users/afarinaqs/Library/CloudStorage/Box-Box/Home Folder aa714/Private/BME LAB/Afi/OOD Detection/Data/20250522"


In [ ]:
def read_summary_files_by_slide_and_sheet(folder):
    data_dict = {}
    for file in glob(os.path.join(folder, '*_N*_summary.xlsx')):
        match = re.search(r'_N(\w+)_summary\.xlsx$', os.path.basename(file))
        if not match:
            continue
        slide = match.group(1)
        xls = pd.ExcelFile(file)
        for sheet in xls.sheet_names:
            df = xls.parse(sheet)
            df['dAb'] = sheet.split('_')[0]
            df['target'] = sheet.split('_')[1]
            data_dict[(slide, sheet)] = df
    return data_dict


all_data = read_summary_files_by_slide_and_sheet(path)
print(all_data.keys())
all_data[('5', 'dAb35_Beta')]


In [ ]:
def extract_cAb_columns(df, dAb, cAb):
    base = f"{dAb}_{cAb}"
    return [col for col in df.columns if col.startswith(base)]

def query_data(data_dict, cAbs, slides, dAb=None, target=None):
    result_rows = []
    result_frames = []
    for (slide_num, sheet_name), df in data_dict.items():
        if slide_num == '13o':
            target = 'Beta'
        else:
            target = 'Delta'
        # print('going for:', slide_num, sheet_name)
        if slide_num not in slides:
            # print(f"{slide_num} is not in {slides}")
            continue
        if dAb and not sheet_name.startswith(dAb):
            # print(f"{dAb} not in {sheet_name}")
            continue
        if target and not sheet_name.endswith(target):
            # print(f"{target} not in {sheet_name}")
            continue

        dAb_val = df['dAb'].iloc[0]
        target_val = df['target'].iloc[0]
        print(target_val)


        for cAb in cAbs:
            cAb_cols = extract_cAb_columns(df, dAb_val, cAb)
        # print(cAb_cols)
        # if not cAb_cols:
        #     continue
            base = f"{slide_num}_cAb{cAb}"
            # print(base)
            temp_df = pd.DataFrame({
                'Ag_Conc': df['Ag_Conc.'],
                'target': target_val.split('_')[-1],
                'dAb': dAb_val,
                f"{base}": df[cAb_cols[0]].values if len(cAb_cols) > 0 else None,
                f"{base}_1": df[cAb_cols[1]].values if len(cAb_cols) > 1 else None,
                f"{base}_2": df[cAb_cols[2]].values if len(cAb_cols) > 2 else None,
            })
            result_frames.append(temp_df)

    if not result_frames:
        return pd.DataFrame()

    # Merge on Ag_Conc, label, and dAb
    merged_df = reduce(
        lambda left, right: pd.merge(left, right, on=['Ag_Conc', 'target', 'dAb'], how='outer'),
        result_frames
    )
    merged_df.drop(columns=['target', 'dAb'], inplace=True)
    return merged_df


cAbs_vec=['55+','71-','41+']
cAbs_vec=['25-','30-','34+']
# cAbs_vec=['29-','59+','36+']
df_result = query_data(all_data, cAbs=cAbs_vec, slides=['11o', '1'], dAb='dAb36', target='Delta')

df_result


In [ ]:
all_data.keys()

In [ ]:
all_data[( '1', 'dAb36_Delta')]
